<a href="https://colab.research.google.com/github/momoash04/Path_Planning-Shell-auc-/blob/main/Shell_Sector_1_Path_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Node class for A* search
class Node:
    def __init__(self, position, parent=None, g=0, h=0, speed=0):
        self.position = position
        self.parent = parent
        self.g = g  # Cost from start (including energy consumption)
        self.h = h  # Heuristic (to goal)
        self.f = g + h
        self.speed = speed  # Current speed

    def __lt__(self, other):
        return self.f < other.f

In [ ]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev
from matplotlib.path import Path

In [ ]:
# Manhattan distance heuristic
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [ ]:
def is_inside_track(point, track_boundaries):
    return Path(track_boundaries).contains_point(point)

In [ ]:
# Movement cost considering speed, energy efficiency, and boundary avoidance
def movement_cost(current, next, track_bounds, max_speed=25, boundary_buffer=2):
    dx, dy = next[0] - current[0], next[1] - current[1]
    speed = np.hypot(dx, dy)  # Calculate movement speed

    # Penalize exceeding speed limit
    if speed > max_speed:
        return float('inf')

    # Energy-efficient cost: penalize sharp turns
    turn_penalty = abs(dx) + abs(dy) > 1  # Diagonal movement

    # Penalize getting too close to boundaries
    min_distance = min(np.hypot(next[0] - x, next[1] - y) for x, y in track_bounds)
    boundary_penalty = 10 if min_distance < boundary_buffer else 0

    return speed + (5 if turn_penalty else 0) + boundary_penalty

In [ ]:
# Get neighbors with energy and speed considerations
def get_neighbors(position, track_boundaries):
    x, y = position
    moves = [(x + dx, y + dy) for dx, dy in [
        (-1, 0), (1, 0), (0, -1), (0, 1),
        (-1, -1), (-1, 1), (1, -1), (1, 1)
    ]]

    return [move for move in moves if is_inside_track(move, track_boundaries)]

In [ ]:
# Calculate target velocity for each path point
def calculate_target_velocity(path, max_speed=25):
    velocities = []
    for i in range(1, len(path)):
        prev_point = path[i - 1]
        current_point = path[i]

        # Calculate distance between points
        distance = np.hypot(current_point[0] - prev_point[0], current_point[1] - prev_point[1])

        # Adjust velocity to maintain smooth movement and respect the speed limit
        velocity = min(distance * 100, max_speed)
        velocities.append(velocity)

    return velocities

In [ ]:
# Apply spline to smooth the path
def smooth_path(path, smoothing_factor=3):
    x, y = zip(*path)
    tck, u = splprep([x, y], s=smoothing_factor)
    u_new = np.linspace(0, 1, len(path) * 10)
    smooth_x, smooth_y = splev(u_new, tck)
    return list(zip(smooth_x, smooth_y))

In [ ]:
# A* search with energy-aware path planning and boundary avoidance
def astar(start, goal, track_bounds, max_speed=25):
    open_list = []
    closed_set = set()
    g_cost = {start: 0}

    start_node = Node(start, None, 0, heuristic(start, goal))
    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        # If goal reached, reconstruct path
        if current_node.position == goal:
            path = []
            total_cost = current_node.g
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            print(f"Total Cost: {total_cost}")
            return path[::-1]

        closed_set.add(current_node.position)

        for neighbor in get_neighbors(current_node.position, track_bounds):
            if neighbor in closed_set:
                continue

            # Calculate new movement cost
            g = current_node.g + movement_cost(current_node.position, neighbor, track_bounds, max_speed)

            if neighbor not in g_cost or g < g_cost[neighbor]:
                g_cost[neighbor] = g
                h = heuristic(neighbor, goal)
                neighbor_node = Node(neighbor, current_node, g, h)
                heapq.heappush(open_list, neighbor_node)

    return []


In [ ]:
# Find the farthest goal point within track
def find_farthest_goal(car_location, track_bounds):
    max_distance = -1
    best_goal = car_location

    min_x = min(x for x, _ in track_bounds)
    max_x = max(x for x, _ in track_bounds)
    min_y = min(y for _, y in track_bounds)
    max_y = max(y for _, y in track_bounds)

    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            candidate = (x, y)
            if is_inside_track(candidate, track_bounds):
                distance = heuristic(car_location, candidate)
                if distance > max_distance:
                    max_distance = distance
                    best_goal = candidate

    return best_goal

In [ ]:
# Visualize track, car, and path
def visualize_path(track_bounds, car_location, goal, path, velocities):
    fig, ax = plt.subplots()

    # Draw track boundaries
    x_vals, y_vals = zip(*track_bounds)
    ax.plot(x_vals + (x_vals[0],), y_vals + (y_vals[0],), 'b-', label='Track Boundaries')

    # Draw car and goal
    ax.plot(car_location[0], car_location[1], 'go', label='Start (Car)')
    ax.plot(goal[0], goal[1], 'ro', label='Goal')

    # Draw smoothed path
    if path:
        px, py = zip(*path)
        ax.plot(px, py, 'y-', label='Smoothed Path')

    # Display velocities every 10m along the path
    distance = 0
    for i in range(1, len(path)):
        prev_point = path[i - 1]
        current_point = path[i]
        segment_length = np.hypot(current_point[0] - prev_point[0], current_point[1] - prev_point[1])
        distance += segment_length

        if distance >= 10:
            ax.text(current_point[0], current_point[1], f"{velocities[min(i, len(velocities) - 1)]:.1f} m/s", fontsize=8, color='red')
            distance = 0

    plt.legend()
    plt.show()

In [ ]:
# Input Data
track_bounds = [(5, 5), (10, 30), (75, 70), (70, 60), (40, 30), (40, 45)]
car_location = (10, 15)

In [ ]:
# Find the optimized goal
goal = find_farthest_goal(car_location, track_bounds)

# Run energy-efficient A* search
path = astar(car_location, goal, track_bounds)

# Smooth the path
smoothed_path = smooth_path(path)

# Calculate target velocities
velocities = calculate_target_velocity(smoothed_path)

# Visualize results
# visualize_path(track_bounds, car_location, goal, smoothed_path, velocities)

Total Cost: 148.8284271247462
